In [1]:
import os
import json
import random
import collections
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from PIL import Image
import numpy as np
import zipfile
 
import config

In [3]:
from data_loader import *

In [4]:
image_path_to_caption = collections.defaultdict(list)
mode = "total"
anno_type = "caption"
top_k = 5000
caption_vector = []
img_name_vector = []

for key, val in category.items():
    for v in val:
        json_dir = os.path.join("./data/DGU-AI-LAB-Korean-Tourist-Spot-Dataset-62a483a/kts//" + mode, key, v, v + ".json")
        image_folder_dir = os.path.join(
            "./data/DGU-AI-LAB-Korean-Tourist-Spot-Dataset-62a483a/kts//" + mode, key, v, "images")

        with open(json_dir, encoding="UTF-16") as f:
            js = json.load(f)
        f.close()

        for j in js:
            image_file_dir = os.path.join(
                image_folder_dir, j["img_name"] + ".jpg")
            # image = Image.open(image_file_dir)
            # j["image"]   = image
            if anno_type=="caption":
                image_path_to_caption[image_file_dir] = f"<start> {j['text']} <end>"
            else:
                image_path_to_caption[image_file_dir] = f"<start> {j['hashtag']} <end>"

        image_paths = list(image_path_to_caption.keys())
                
        random.shuffle(image_paths)


In [6]:
for image_path in image_paths:
    caption = image_path_to_caption[image_path]
    caption_vector.append(caption)
    img_name_vector.append(image_path)

print("total Set : ", len(caption_vector))

total Set :  4000


In [9]:
# Get unique images
encode_train = sorted(set(img_name_vector))

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)

In [73]:
import numpy 

for item in image_dataset:
    print(item.numpy().decode('utf-8'))
    break

./data/DGU-AI-LAB-Korean-Tourist-Spot-Dataset-62a483a/kts//total\nature-scene\beach\images\1.jpg


In [31]:
image_dataset = encode_train.map(
    load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

AttributeError: 'list' object has no attribute 'map'

In [ ]:
for img, path in image_dataset:
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
        (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        print(p)
        path_of_feature = p.numpy().decode("utf-8")
        np.save(path_of_feature, bf.numpy())


In [ ]:

# Choose the top 5000 words from the vocabulary
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                oov_token="<unk>",
                                                filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_list)
train_seqs = tokenizer.texts_to_sequences(caption_list)

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')


# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)


img_to_cap_vector = collections.defaultdict(list)
for img, cap in zip(img_name_vector, cap_vector):
    img_to_cap_vector[img].append(cap)

# Create training and validation sets using an 80-20 split randomly.
img_keys = list(img_to_cap_vector.keys())
random.shuffle(img_keys)

slice_index = int(len(img_keys)*0.8)
img_name_train_keys, img_name_val_keys = img_keys[:slice_index], img_keys[slice_index:]

img_name_train = []
cap_train = []
for imgt in img_name_train_keys:
    capt_len = len(img_to_cap_vector[imgt])
    img_name_train.extend([imgt] * capt_len)
    cap_train.extend(img_to_cap_vector[imgt])

print("train set : ", len(cap_train))

img_name_val = []
cap_val = []
for imgv in img_name_val_keys:
    capv_len = len(img_to_cap_vector[imgv])
    img_name_val.extend([imgv] * capv_len)
    cap_val.extend(img_to_cap_vector[imgv])

print("validation set : ", len(cap_val))

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
        map_func, [item1, item2], [tf.float32, tf.int32]),
        num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

print("data set load complete!")

In [24]:
encoder = CNN_Encoder(config.embedding_dim)
decoder = RNN_Decoder(config.embedding_dim, config.units, config.vocab_size)

zip file is ready but not uncompressed.
start uncompress..
uncompress success


In [ ]:
Trainer()